In [ ]:
class trade_condition:
    def __init__(self):
        self.__orders = {}
        self.__printable = None
        self.__non_printable = []

    def snapshot(self, iter):
        for obj in iter:
            order_id = obj["order_id"]
            self.__orders[order_id] = obj["buy"]

    def __flush(self):
        if self.__printable is not None:
            self.__printable["trade_condition"] = "TSUM"
            yield self.__printable
            first = True
            for non_printable in self.__non_printable:
                order_id = non_printable["order_id"]
                buy = self.__printable["buy"]
                if not first:
                    buy = not buy
                if order_id in self.__orders.keys():
                    buy = self.__orders[order_id]
                first = False
                if (
                    buy == self.__printable["buy"]
                    and non_printable["quantity"] == self.__printable["quantity"]
                ):
                    if buy:
                        non_printable["trade_condition"] = "AB"
                    else:
                        non_printable["trade_condition"] = "AS"
                else:
                    non_printable["trade_condition"] = "OR"
                yield non_printable
            self.__printable = None
            self.__non_printable = []

    def trades(self, iter):
        for obj in iter:
            if obj["type"] == "add_order":
                order_id = obj["order_id"]
                self.__orders[order_id] = obj["buy"]
                continue
            if obj["type"] == "delete_order":
                order_id = obj["order_id"]
                if order_id in self.__orders.keys():
                    del self.__orders[obj["order_id"]]
                continue
            if obj["type"] == "trade":
                if obj["printable"]:
                    for out in self.__flush():
                        yield out
                    self.__printable = obj
                    continue
                self.__non_printable.append(obj)


def take(n, iter):
    i = 0
    for obj in iter:
        if i == n:
            break
        yield obj
        i += 1

In [ ]:
import hpq
import ijson
import IPython
import tabulate

snapshot = {
    "query": "snapshot",
    "source": "cme",
    "product": "ESZ2",
    "date": "2022-10-27",
    "time": "05:00:00",
    "type": "orders",
    "time_zone": "UTC",
    "timestamp_mode": "exchange",
}
trades = {
    "query": "ticks",
    "messages": ["TRD"],
    "source": "cme",
    "product": snapshot["product"],
    "date": snapshot["date"],
    "start_time": snapshot["time"],
    "end_time": "05:15:00",
    "time_zone": snapshot["time_zone"],
    "timestamp_mode": snapshot["timestamp_mode"],
}
connection = hpq.create_web_socket_client()
state = trade_condition()
state.snapshot(ijson.items(connection.stream(snapshot), "item"))
rows = list(
    map(
        hpq.format,
        take(50, state.trades(ijson.items(connection.stream(trades), "item"))),
    )
)
table = tabulate.tabulate(rows, tablefmt="html", headers="keys")
IPython.display.display(IPython.display.HTML(table))